In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## Install libraries

```bash
conda create -n edu4 python=3.11 jupyter matplotlib
```

```bash 
! pip install -U -r requirements.txt
```

```bash
! pip install -U numpy
! pip install -U scikit-learn
```

## Update repository

In [ ]:
# ! git pull

## Add import path

In [ ]:
import gc
import os
import sys

In [ ]:
def add_library_level(level=4):
    suf_path = ['..']
    path = '..'
    for i in range(0, level):
        join_path = suf_path * i
        path = '/'.join(join_path)
        module_path = os.path.abspath(os.path.join(path))
        if module_path not in sys.path:
            sys.path.append(module_path)
            print(f'Appendeding {path}')

In [ ]:
add_library_level(level=4)

## Organize imports

In [ ]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
from huggingface_hub import hf_hub_download, notebook_login
import numpy as np
import torch
from torch import nn

In [ ]:
from sae_lens import SAE, HookedSAETransformer
from transformer_lens.utils import tokenize_and_concatenate

In [ ]:
from transformer_lens import HookedTransformer

In [ ]:
from datasets import load_dataset

In [ ]:
import multiprocessing
from pathlib import Path

In [ ]:
from tqdm import tqdm

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
import plotly.express as px

In [ ]:
from src.lattmc.fca.utils import *
from src.lattmc.fca.data_utils import *
from src.lattmc.fca.image_utils import *
from src.lattmc.fca.models import *
from src.lattmc.fca.fca_utils import *
from src.lattmc.fca.image_gens import *

In [ ]:
from src.lattmc.sae.nlp_sae_utils import *

In [ ]:
import logging

#### Number of CPU cores

In [ ]:
workers = multiprocessing.cpu_count()
workers

In [ ]:
SEED = 2024

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
device = init_device()

In [ ]:
torch.__version__

In [ ]:
np.__version__

## Initialize Layer

In [ ]:
layer = 10
dataset_suffix = 'pile'

## Initialize Path

In [ ]:
def init_data(model_name, layer, dataset_suffix='pile'):
    PATH = Path('../data')
    dataset_suffix = 'pile'
    vectors_name = f'{model_name.replace('-', '_')}_{layer}_{dataset_suffix}'
    checkpoint_dir = PATH / 'saes'
    vectors_dir = checkpoint_dir / f'{vectors_name}_vecs'
    matrix_dir = checkpoint_dir / f'{vectors_name}_mats'
    vectors_path = checkpoint_dir / f'{vectors_name}_vecs.joblib'
    
    mkdirs(
        checkpoint_dir,
        vectors_dir,
        matrix_dir
    )
    logger.info(f'{matrix_dir = } {vectors_path = }')

    return matrix_dir, vectors_path

## Initialize simple dataset

In [ ]:
dataset = load_dataset(
    path='NeelNanda/pile-10k',
    split='train',
    streaming=False,
)

## Initialize model

In [ ]:
model_name = 'gpt2-small'
release = 'gpt2-small-res-jb'
sae_id = f'blocks.{layer}.hook_resid_pre'

In [ ]:
matrix_dir, vector_path = init_data(model_name, layer, dataset_suffix=dataset_suffix)

In [ ]:
net = Text2Sae(
    model_name,
    release,
    sae_id,
    device
)

## Generate V Lattice

In [ ]:
gc.collect()

In [ ]:
matrix_dir

In [ ]:
init_matrices(matrix_dir, dataset, net)

In [ ]:
gc.collect()

In [ ]:
V = init_vectors(vector_path, matrix_dir, segment=False)

## Generate Context and Analyze

In [ ]:
gc.collect()

In [ ]:
fca = FCA(V)

In [ ]:
text1 = "The Golden Gate Bridge"
text2 = "The Brooklyn Bridge"
text3 = "The card game Bridge"

In [ ]:
sent1 = 'New York City'
sent2 = 'Golden Gate Bridge'
sent3 = 'Grand Canyon Park'

In [ ]:
word_mapper = WordMapper(net, dataset, matrix_dir)

In [ ]:
cu = ConceptUtils(fca, word_mapper)

## New York Ciry

In [ ]:
v_ns = net.embed(sent1)[0][1:]
v_gs = net.embed(sent2)[0][1:]

In [ ]:
t_ns = net.tokenize(sent1)[0][1:]
t_gs = net.tokenize(sent2)[0][1:]

In [ ]:
t_ns.shape, t_gs.shape

In [ ]:
t_ns, t_gs

In [ ]:
topK(v_ns[0], 10)

In [ ]:
topK(v_gs[0], 10)

In [ ]:
m_bg = meet(v_ns[2], v_gs[2])
topK(m_bg, 20)

In [ ]:
print(f'Intersection of {net.to_string(t_ns[2])} and {net.to_string(t_gs[2])}')

In [ ]:
m_cg, c_bg, w_bg = cu.gen_print(19016, 10)

In [ ]:
w_bg

#### Neighboor Analysis

In [ ]:
topK(c_bg.v, 20)

In [ ]:
n_cg, c_nbg, w_nbg = cu.gen_print(6863, 8, context=[8, 8])

In [ ]:
w_nbg

In [ ]:
c_mbg = c_bg & c_nbg
c_mbg

In [ ]:
topK(c_mbg.v, 20)

In [ ]:
m_vg, c_vmg, w_vgm = cu.gen_print(23992, 29, context=[8, 8])

In [ ]:
w_vgm

## Experiment with Tokenizer

In [ ]:
?? net.model.tokenizer.encode

#### Neighboor Analysis for Golden

In [ ]:
def gen_neughbors(idxs, vals, context=[8, 8]):
    v_idx, c_idx = gen_concept(idxs, vals)
    words = word_mapper(c_idx, v=v_idx, context=context)

    return v_idx, c_idx, words   

In [ ]:
def gen_print(idxs, vals, context=[8, 8]):
    v_idx, c_idx, words = gen_neughbors(idxs, vals, context=context)
    print(f'{c_idx = }')
    print('\n'.join(f'{wd}' for wd in enumerate(words)))

    return v_idx, c_idx, words

In [ ]:
printTopK(v_gs[0], k=20)

In [ ]:
n_gg, c_gg, w_gg = cu.gen_print(19837, 79, context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([19837, 21286], [8, 2], context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([21286, 4507], [15, 12], context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([4507, 14717], [12, 5], context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([4507, 14717, 6183], [12, 5, 5], context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([14717, 6183], [5, 5], context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([6183, 332, 2407], [5, 3, 2], context=[8, 8])

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([332, 2407, 10363], [3, 2, 2], context=[8, 8])

In [ ]:
printTopK(c_gg.v, 20)

In [ ]:
n_gn, c_gn, w_gn = cu.gen_print(6284, 12, context=[8, 8])

In [ ]:
c_mbg = c_bg & c_nbg
c_mbg

In [ ]:
printTopK(c_mbg.v, 20)

In [ ]:
m_vg, c_vmg, w_vgm = cu.gen_print(23992, 29, context=[8, 8])

## Apple Case

In [ ]:
# Example inputs: same word "apple" in two different contexts
text1 = "I ate an apple for breakfast."
text2 = "Apple Inc. unveiled its latest product."

In [ ]:
v_a1 = net.embed(text1)[0][1:]
v_a2 = net.embed(text2)[0][1:]

In [ ]:
t_a1 = net.tokenize(text1)[0][1:]
t_a2 = net.tokenize(text2)[0][1:]

In [ ]:
v_a1.shape, t_a1.shape, v_a2.shape, t_a2.shape, 

In [ ]:
for i in range(t_a1.shape[0]): 
    print(f'{i} {net.to_string(t_a1[i])}, {net.to_string(t_a2[i])}\n')

In [ ]:
printTopK(v_a1[3], k=20)

In [ ]:
v_am = meet(v_a1[3], v_a2[0])
printTopK(v_am)

In [ ]:
v_2gg, c_2gg, ws_gg = cu.gen_print([4269, 4809, 23638], [30, 1, 1], context=[8, 8])

In [ ]:
printTopK(v_a1[1])

In [ ]:
v_ate, c_ate, ws_ate = cu.gen_print([15767], [51], context=[8, 8])

In [ ]:
v_ate, c_ate, ws_ate = cu.gen_print([9493, 22952], [18, 8], context=[8, 8])

In [ ]:
printTopK(v_a2[1])

In [ ]:
v_ate, c_ate, ws_ate = cu.gen_print([23563], [65], context=[8, 8])

In [ ]:
v_ate, c_ate, ws_ate = cu.gen_print([9768, 18294], [16 , 11], context=[8, 8])

In [ ]:
v_apple_inc = meet(v_a2[0], v_a2[1])
printTopK(v_apple_inc)

In [ ]:
v_aplle_inc, c_aplle_inc, ws_aplle_inc = cu.gen_print([17725, 7574], [9 , 6], context=[8, 8])

In [ ]:
v_apple_ate = meet(v_a1[1], v_a1[3])
printTopK(v_apple_ate)

In [ ]:
val_apple_inc, idx_apple_ink = topK(v_apple_inc, 20)
val_apple_ate, idx_apple_ate = topK(v_apple_ate, 20)

In [ ]:
val_apple_inc == val_apple_ate

In [ ]:
val_apple_inc, val_apple_ate

In [ ]:
idx_apple_ink == idx_apple_ate

In [ ]:
idx_apple_ink, idx_apple_ate

In [ ]:
v_aplle_ate, c_aplle_ate, ws_aplle_ate = cu.gen_print([11930, 19398], [6 , 2], context=[8, 8])

In [ ]:
v_aplle_ate, c_aplle_ate, ws_aplle_ate = cu.gen_print([19398, 22952], [2 , 2], context=[8, 8])

## Sentence generation

In [ ]:
c_golden = fca.G_FG(v_gs[0] / 10)
c_gate = fca.G_FG(v_gs[1] / 10)
c_bridge = fca.G_FG(v_gs[2] / 10)

In [ ]:
c_golden, c_gate, c_bridge

In [ ]:
c_golden_gate = c_golden & c_gate
c_golden_gate

In [ ]:
dataset[c_golden_gate.A[0].item()]['text']

In [ ]:
ws = word_mapper(c_golden_gate, context=True)

In [ ]:
ws

In [ ]:
v_nyc = net.embed(sent1)[0][1:]
v_ggb = net.embed(sent2)[0][1:]
v_gcp = net.embed(sent3)[0][1:]
v_nyc.shape, sent1, v_ggb.shape, sent2, v_gcp.shape, sent3

In [ ]:
join_all(v_ggb).shape

In [ ]:
c_ggb = fca.G_FG(v_ggb[1] / 10)
c_ggb

In [ ]:
dataset[c_ggb.A[2].item()]['text']

In [ ]:
ws = word_mapper(c_ggb, v=v_ggb[0] / 100)

In [ ]:
ws

In [ ]:
topK(v_gcp[0], 20)

In [ ]:
v_gcm = meet(v_gcp[0], v_gcp[1])
topK(v_gcm, 10)

In [ ]:
v_ggm = meet(v_ggb[0], v_ggb[1])
topK(v_ggm, 10)

In [ ]:
c_ggm = fca.G_FG(v_ggm)

In [ ]:
w_ggm = word_mapper(c_ggm)
w_ggm

In [ ]:
items = dataset[c_ggm.A.tolist()]

In [ ]:
items

In [ ]:
v_i, cn_i = gen_concept(7525, 6)
cn_i

In [ ]:
ws = word_mapper(cn_i, v=v_i, context=True)

In [ ]:
ws